In [1]:
import pandas as pd

In [2]:
data_firm_level = pd.read_stata(r"..\Data\Firm_patent\data_firm_level.dta")

In [3]:
data_firm_level.head()

,year,gvkey,fyr,csho,ebit,ebitda,emp,pstk,sale,xrd,...,rivao,rintan,ract,value,value_e,value_d,qkstock,tobinq,tobinq_e,tobinq_d
0,1994.0,10846,12.0,363.661,3856.000,4977.000,304.000,NaN,45419.000,831.000,...,148.300720,0.000000,15795.056641,NaN,NaN,-10743.563477,19477.857422,NaN,NaN,-0.551578
1,1995.0,10846,12.0,363.800,3986.000,5242.000,308.000,NaN,49732.000,923.000,...,169.000000,0.000000,16314.000000,NaN,NaN,-11067.000000,20419.000000,NaN,NaN,-0.541995
2,1996.0,10846,12.0,363.816,4473.000,5794.000,306.000,153.435,52161.000,934.000,...,187.683273,0.000000,16892.472656,51197.640625,62475.253906,-11277.615234,19883.673828,2.574858,3.142038,-0.567180
3,1996.0,153351,12.0,15.669,-3.810,-3.766,0.012,0.001,2.000,3.344,...,0.000000,0.000000,47.806450,124.507561,172.314011,-47.806450,0.085044,1464.037354,2026.175293,-562.137939
4,1996.0,27961,12.0,NaN,15.514,20.698,0.915,0.000,170.821,0.900,...,0.000000,144.434006,57.626587,NaN,NaN,98.776138,204.719437,NaN,NaN,0.482495


In [4]:
data_firm_level.describe()

,year,gvkey,fyr,csho,ebit,ebitda,emp,pstk,sale,xrd,...,rivao,rintan,ract,value,value_e,value_d,qkstock,tobinq,tobinq_e,tobinq_d
count,12987.000000,12987.000000,12987.000000,12972.000000,12987.000000,12959.000000,12987.000000,12984.000000,12987.000000,12987.000000,...,12249.000000,12987.000000,12863.000000,12830.000000,12955.000000,12862.000000,11092.000000,10954.000000,1.105200e+04,10985.000000
mean,2004.331023,56854.403480,10.026334,233.957002,588.087075,890.304981,14.793719,5.594837,5033.167998,234.863520,...,313.752075,962.701477,1666.342529,4952.295898,5887.724121,-655.460754,2973.737061,23.008007,2.859210e+01,-5.800142
std,3.404230,58906.719922,3.233852,739.313699,2444.137458,3469.930124,64.137153,63.099025,19075.794789,815.738421,...,3014.001953,5316.743652,5330.618652,18031.783203,21045.695312,3363.937988,13321.155273,636.676392,7.020701e+02,90.511459
min,1994.000000,1013.000000,1.000000,0.001000,-10537.000000,-5062.000000,0.000000,-109.000000,-0.294000,-0.100000,...,0.000000,0.000000,0.005036,-9883.476562,0.000015,-56679.519531,0.000000,-1510.426758,1.849711e-07,-6249.000488
25%,2001.000000,9555.000000,9.000000,21.664750,-9.135000,-3.541000,0.251000,0.000000,51.786000,8.181000,...,0.000000,0.831040,49.265190,58.652008,114.832901,-323.524628,17.262848,0.868188,1.472013e+00,-2.302479
50%,2004.000000,28194.000000,12.000000,44.654000,14.081000,30.055000,1.119000,0.000000,283.267000,27.436000,...,0.000000,24.231129,182.555359,311.854706,456.005768,-86.995613,95.920517,2.141841,3.271380e+00,-0.945903
75%,2007.000000,112040.000000,12.000000,125.452750,175.212500,278.050000,7.347000,0.000000,1808.393000,91.027000,...,16.659876,265.783340,793.660278,1763.491455,2294.695312,-18.521852,690.134949,6.176023,8.552984e+00,-0.329962
max,2010.000000,282553.000000,12.000000,10862.000000,50182.000000,62540.000000,2100.000000,2500.000000,420016.000000,12183.000000,...,129072.296875,147387.968750,117406.539062,403882.062500,462959.437500,50982.308594,268091.593750,63620.855469,6.986986e+04,589.391296


In [5]:
patents_firm_merge = pd.read_stata(r"..\Data\Firm_patent\patents_firm_merge.dta")

In [6]:
patents_firm_merge.head()

,index,patnum,fdate,idate,pdate,permno,patent_class,subclass,ncites,xi,year,Npats,Tcw,Tsm,tcw,tsm,_merge
0,37352,1605417,10/23/1923,11/02/1926,,10006.0,403.0,206000O,2.0,0.046886,1926,10,18.980768,0.375693,NaN,NaN,matched (3)
1,11188,1579234,05/14/1923,04/06/1926,,10006.0,74.0,503000O,4.0,0.031358,1926,10,18.980768,0.375693,NaN,NaN,matched (3)
2,37345,1605410,04/09/1926,11/02/1926,,10006.0,295.0,042000O,1.0,0.046886,1926,10,18.980768,0.375693,NaN,NaN,matched (3)
3,37377,1605442,03/24/1922,11/02/1926,,10006.0,164.0,168000O,0.0,0.046886,1926,10,18.980768,0.375693,NaN,NaN,matched (3)
4,37350,1605415,01/22/1923,11/02/1926,,10006.0,267.0,086000O,2.0,0.046886,1926,10,18.980768,0.375693,NaN,NaN,matched (3)


In [7]:
patents_firm_merge.describe()

,index,patnum,permno,ncites,xi,year,Npats,Tcw,Tsm,tcw,tsm
count,1.844881e+06,1.844881e+06,1.844881e+06,1.801879e+06,1.801879e+06,1.844881e+06,1.844881e+06,1.844881e+06,1.844881e+06,1.669418e+06,1.669418e+06
mean,3.579055e+06,5.156201e+06,3.466204e+04,1.025662e+01,1.629867e+01,1.986901e+03,5.552485e+02,1.173347e+03,6.289900e+03,1.582112e-01,3.115258e-01
std,1.659959e+06,1.666433e+06,2.549417e+04,2.013455e+01,5.985516e+01,1.998026e+01,8.358954e+02,1.781197e+03,1.733616e+04,4.999385e-01,5.596020e-01
min,8.400000e+01,1.568124e+06,1.000600e+04,0.000000e+00,0.000000e+00,1.926000e+03,1.000000e+00,1.000000e+00,0.000000e+00,4.921287e-07,0.000000e+00
25%,2.205231e+06,3.777131e+06,1.409000e+04,1.000000e+00,4.743302e-01,1.973000e+03,5.600000e+01,1.149980e+02,7.486781e+01,2.816062e-02,3.964551e-02
50%,3.658791e+06,5.233054e+06,2.234900e+04,5.000000e+00,2.941844e+00,1.993000e+03,2.220000e+02,4.673977e+02,5.376674e+02,6.449876e-02,1.495097e-01
75%,5.078303e+06,6.662437e+06,5.372700e+04,1.100000e+01,1.177292e+01,2.003000e+03,6.290000e+02,1.385416e+03,3.596585e+03,1.460098e-01,3.382327e-01
max,6.240105e+06,7.827616e+06,9.328700e+04,2.424000e+03,6.208359e+03,2.010000e+03,5.052000e+03,1.016166e+04,2.595804e+05,7.655125e+01,2.226794e+01


In [8]:
data_patent_level = pd.read_stata(r"..\Data\Patent_level_data\data_patent_level.dta")

In [9]:
data_patent_level.head()

,patnum,fdate,idate,pdate,permno,patent_class,subclass,ncites,xi,year,Npats,Tcw,Tsm,forward_citations,backward_citations
0,1568124,06/12/1920,01/05/1926,,12503.0,56.0,006000O,0.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
1,1568134,09/09/1922,01/05/1926,,12503.0,239.0,584000O,0.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
2,1568142,08/13/1920,01/05/1926,,10401.0,333.0,168000O,1.0,1.625674,1926,430,871.180249,187.894016,NaN,1.0
3,1568163,05/03/1920,01/05/1926,,12503.0,56.0,011500O,1.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
4,1568338,11/02/1923,01/05/1926,,14875.0,400.0,689000O,3.0,0.163435,1926,104,134.871391,5.036761,NaN,3.0


In [10]:
data_patent_level.describe()

,patnum,permno,ncites,xi,year,Npats,Tcw,Tsm,forward_citations,backward_citations
count,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.531473e+06,1.502493e+06
mean,5.167546e+06,3.484628e+04,1.025952e+01,1.629619e+01,1.986999e+03,5.628692e+02,1.189529e+03,6.391305e+03,1.088757e+01,1.190791e+01
std,1.669579e+06,2.552283e+04,2.013693e+01,5.985719e+01,2.005846e+01,8.414713e+02,1.792490e+03,1.749233e+04,2.161838e+01,2.145773e+01
min,1.568124e+06,1.000600e+04,0.000000e+00,0.000000e+00,1.926000e+03,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,3.794678e+06,1.409000e+04,1.000000e+00,4.742682e-01,1.974000e+03,5.700000e+01,1.167448e+02,7.528275e+01,3.000000e+00,2.000000e+00
50%,5.261326e+06,2.256800e+04,5.000000e+00,2.940681e+00,1.993000e+03,2.260000e+02,4.795861e+02,5.444828e+02,6.000000e+00,6.000000e+00
75%,6.674794e+06,5.418100e+04,1.100000e+01,1.176838e+01,2.004000e+03,6.420000e+02,1.429433e+03,3.657433e+03,1.100000e+01,1.300000e+01
max,7.827616e+06,9.328700e+04,2.424000e+03,6.208359e+03,2.010000e+03,5.052000e+03,1.016166e+04,2.595804e+05,1.176000e+03,2.413000e+03


In [25]:
data_patent_level = pd.read_stata(r"..\Data\Patent_level_data\data_patent_level.dta")

In [12]:
data_patent_level.head()

,patnum,fdate,idate,pdate,permno,patent_class,subclass,ncites,xi,year,Npats,Tcw,Tsm,forward_citations,backward_citations
0,1568124,06/12/1920,01/05/1926,,12503.0,56.0,006000O,0.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
1,1568134,09/09/1922,01/05/1926,,12503.0,239.0,584000O,0.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
2,1568142,08/13/1920,01/05/1926,,10401.0,333.0,168000O,1.0,1.625674,1926,430,871.180249,187.894016,NaN,1.0
3,1568163,05/03/1920,01/05/1926,,12503.0,56.0,011500O,1.0,0.161681,1926,15,25.664662,4.485454,NaN,NaN
4,1568338,11/02/1923,01/05/1926,,14875.0,400.0,689000O,3.0,0.163435,1926,104,134.871391,5.036761,NaN,3.0


In [13]:
data_patent_level.describe()

,patnum,permno,ncites,xi,year,Npats,Tcw,Tsm,forward_citations,backward_citations
count,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.801308e+06,1.531473e+06,1.502493e+06
mean,5.167546e+06,3.484628e+04,1.025952e+01,1.629619e+01,1.986999e+03,5.628692e+02,1.189529e+03,6.391305e+03,1.088757e+01,1.190791e+01
std,1.669579e+06,2.552283e+04,2.013693e+01,5.985719e+01,2.005846e+01,8.414713e+02,1.792490e+03,1.749233e+04,2.161838e+01,2.145773e+01
min,1.568124e+06,1.000600e+04,0.000000e+00,0.000000e+00,1.926000e+03,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,3.794678e+06,1.409000e+04,1.000000e+00,4.742682e-01,1.974000e+03,5.700000e+01,1.167448e+02,7.528275e+01,3.000000e+00,2.000000e+00
50%,5.261326e+06,2.256800e+04,5.000000e+00,2.940681e+00,1.993000e+03,2.260000e+02,4.795861e+02,5.444828e+02,6.000000e+00,6.000000e+00
75%,6.674794e+06,5.418100e+04,1.100000e+01,1.176838e+01,2.004000e+03,6.420000e+02,1.429433e+03,3.657433e+03,1.100000e+01,1.300000e+01
max,7.827616e+06,9.328700e+04,2.424000e+03,6.208359e+03,2.010000e+03,5.052000e+03,1.016166e+04,2.595804e+05,1.176000e+03,2.413000e+03


In [14]:
cites = pd.read_stata(r"..\Data\Patent_level_data\USPatent_1926-2010\cites\cites.dta")

In [15]:
cites.head()

,citing,cited
0,2364973,1929241
1,2364973,1917995
2,2364973,2157122
3,2364973,2173741
4,2364973,1985654


In [16]:
cites.describe()

,citing,cited
count,5.251991e+07,5.251991e+07
mean,6.182967e+06,4.760499e+06
std,1.444457e+06,1.443307e+06
min,2.364973e+06,4.000000e+00
25%,5.254441e+06,3.916395e+06
50%,6.493971e+06,4.976377e+06
75%,7.381219e+06,5.821890e+06
max,8.087093e+06,8.050926e+06


In [17]:
firm_innovation_v2 = pd.read_stata(r"..\Data\Patent_level_data\USPatent_1926-2010\firm_innovation\firm_innovation_v2.dta")

In [18]:
firm_innovation_v2.head()

,permno,year,npats,Tcw,Tsm,tcw,tsm
0,10006,1926,10,18.980768,0.375693,NaN,NaN
1,10006,1927,17,24.725719,0.803838,NaN,NaN
2,10006,1928,15,34.932781,0.718042,NaN,NaN
3,10006,1929,26,64.189056,1.568269,NaN,NaN
4,10006,1930,16,35.434711,1.932471,NaN,NaN


In [19]:
firm_innovation_v2.describe()

,permno,year,npats,Tcw,Tsm,tcw,tsm
count,65333.000000,65333.000000,65333.000000,65333.000000,65333.000000,5.381100e+04,53811.000000
mean,44737.019806,1982.640105,27.579921,60.017654,449.468628,1.883183e-01,0.118961
std,27466.318539,20.197938,121.514061,261.355774,4003.377686,8.457907e-01,0.327683
min,10006.000000,1926.000000,1.000000,1.000000,0.000000,4.921287e-07,0.000000
25%,19351.000000,1970.000000,1.000000,3.071412,0.840483,1.299177e-02,0.010584
50%,39263.000000,1987.000000,4.000000,8.496572,4.897320,4.620258e-02,0.033290
75%,75696.000000,1999.000000,13.000000,28.587692,38.842308,1.488965e-01,0.105944
max,93287.000000,2010.000000,5052.000000,10161.664062,259580.406250,7.655125e+01,22.267939


In [23]:
patents_xi = pd.read_stata(r"..\Data\Patent_level_data\USPatent_1926-2010\patents_xi\patents_xi.dta")

In [24]:
patents_xi.head()

,patnum,fdate,idate,pdate,permno,class,subclass,ncites,xi
0,3620911,,11/16/1971,,32563.0,,,NaN,NaN
1,3589077,,06/29/1971,,22680.0,,,NaN,NaN
2,3675244,,07/11/1972,,NaN,,,NaN,NaN
3,3703250,,11/21/1972,,NaN,,,NaN,NaN
4,3470564,,10/07/1969,,NaN,,,NaN,NaN


In [26]:
patents_xi.describe()

,patnum,permno,ncites,xi
count,6.240106e+06,1.930089e+06,1.801879e+06,1.801879e+06
mean,4.695703e+06,3.511032e+04,1.025662e+01,1.626178e+01
std,1.807782e+06,2.577234e+04,2.013455e+01,5.970880e+01
min,1.568040e+06,1.000600e+04,0.000000e+00,0.000000e+00
25%,3.129564e+06,1.409000e+04,1.000000e+00,4.743301e-01
50%,4.692958e+06,2.259200e+04,5.000000e+00,2.941844e+00
75%,6.263094e+06,5.578200e+04,1.100000e+01,1.177292e+01
max,7.827616e+06,9.328700e+04,2.424000e+03,6.208359e+03


In [27]:
patent_values = pd.read_stata(r"..\Data\Patent_level_data\Patent_CRSP_match_1929-2017\patent_values\patent_values.dta")

In [28]:
patent_values.head()

,patnum,permco,fdate,idate,xi
0,1571190,22600,,2/2/26,0.233664
1,1571216,22462,10/4/23,2/2/26,0.488463
2,1571234,21912,1/20/22,2/2/26,0.107467
3,1571242,22600,11/1/23,2/2/26,0.233664
4,1571244,22600,3/11/24,2/2/26,0.233664


In [29]:
patent_values.describe()

,patnum,permco,xi
count,2.459480e+06,2.459480e+06,2.459480e+06
mean,5.962815e+06,1.921224e+04,2.422639e+01
std,2.086225e+06,1.054512e+04,6.916226e+01
min,1.571190e+06,4.000000e+00,2.087011e-04
25%,4.207790e+06,1.533800e+04,1.124972e+00
50%,6.096658e+06,2.090800e+04,6.385371e+00
75%,7.578845e+06,2.181400e+04,2.156493e+01
max,9.763369e+06,5.579500e+04,5.340154e+03
